In [1]:
import os
import requests
from datetime import datetime, timedelta
from xml.dom import minidom
import sys
import threading
import pprint

BOE_URL = 'https://boe.es'
BOE_API_SUMARIO = 'https://boe.es/diario_boe/xml.php?id=BOE-S-'
START_DATE = datetime.strptime('20231001', '%Y%m%d')
PATH_DATA = os.path.join('boe', 'dias')
diff_1_day = timedelta(days=1)

# Scrappig BOE

## Funciones

In [2]:
def make_dirs(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        pass

In [3]:
def traer_documento(origen, destino):
    intentos = 0
    max_intentos = 5
    
    while intentos < max_intentos:
        if intentos != 0:
            import time
            time.sleep(5)
            print(f'Intento {intentos}')

        response = requests.get(origen, stream=True)
        if response.status_code == 200:
            print(f'Guardando en: {destino}')
            with open(destino, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            break
        else:
            intentos += 1

In [4]:
def get_pdfs(current_date):
    print('current_date:', current_date.strftime('%Y-%m-%d'))

    fecha_anno, fecha_mes, fecha_dia = current_date.strftime('%Y %m %d').split()

    for dir_path in [os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia),
                    os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'pdfs')]:
        make_dirs(dir_path)

    fichero_sumario_xml = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'index.xml')

    if os.path.exists(fichero_sumario_xml):
        os.remove(fichero_sumario_xml)

    print('Solicitando', f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")} --> {fichero_sumario_xml}')
    traer_documento(f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")}', fichero_sumario_xml)

    tamano_sumario_xml = os.path.getsize(fichero_sumario_xml)
    print('Recibidos:', tamano_sumario_xml, 'bytes')

    if tamano_sumario_xml < 10:
        print('ERROR: Sumario XML erróneo o incompleto')
        sys.exit(1)

    xml_sumario = minidom.parse(fichero_sumario_xml)

    if xml_sumario.documentElement.nodeName == 'error':
        os.remove(fichero_sumario_xml)
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'pdfs'))
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia))
        print('AVISO: No existen boletines para la current_date', current_date.strftime('%Y-%m-%d'))
    else:
        pdfs = xml_sumario.getElementsByTagName('urlPdf')
        print(f"{len(pdfs)} encontrados")
        for pdf in pdfs:
            fichero_pdf = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'pdfs', pdf.firstChild.nodeValue).replace(' ', '_').replace('/', '\\')[1:]
            fichero_pdf_tamano_xml = pdf.getAttribute('szBytes')
            if os.path.exists(fichero_pdf):
                print("fichero encontrado")
                if os.path.getsize(fichero_pdf) == int(fichero_pdf_tamano_xml):
                    continue
                else:
                    try:
                        os.remove(fichero_pdf)
                    except:
                        pass


            print('Solicitando', f'{BOE_URL}{pdf.firstChild.nodeValue} --> {fichero_pdf}')
            intentos = 0
            max_intentos = 5
            while intentos < max_intentos:
                if intentos != 0:
                    import time
                    time.sleep(5)
                    print(f'Intento {intentos}')

                traer_documento(f'{BOE_URL}{pdf.firstChild.nodeValue}', fichero_pdf)
                intentos += 1

                if os.path.getsize(fichero_pdf) == int(fichero_pdf_tamano_xml):
                    break

            if os.path.getsize(fichero_pdf) != int(fichero_pdf_tamano_xml):
                print('ERROR: El tamaño del fichero PDF descargado no coincide con el del XML del Sumario '
                    f'(Descargado: {os.getsize(fichero_pdf)} <> XML: {fichero_pdf_tamano_xml})')
                sys.exit(1)

In [5]:
def get_xmls(current_date, semaphore=None):
    print('current_date:', current_date.strftime('%Y-%m-%d'))

    fecha_anno, fecha_mes, fecha_dia = current_date.strftime('%Y %m %d').split()

    for dir_path in [os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia),
                    os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'xmls')]:
        make_dirs(dir_path)

    fichero_sumario_xml = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'index.xml')

    if os.path.exists(fichero_sumario_xml):
        os.remove(fichero_sumario_xml)

    print('Solicitando', f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")} --> {fichero_sumario_xml}')
    traer_documento(f'{BOE_API_SUMARIO}{current_date.strftime("%Y%m%d")}', fichero_sumario_xml)

    tamano_sumario_xml = os.path.getsize(fichero_sumario_xml)
    print('Recibidos:', tamano_sumario_xml, 'bytes')

    if tamano_sumario_xml < 10:
        print('ERROR: Sumario XML erróneo o incompleto')
        sys.exit(1)

    xml_sumario = minidom.parse(fichero_sumario_xml)

    if xml_sumario.documentElement.nodeName == 'error':
        os.remove(fichero_sumario_xml)
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'xmls'))
        os.rmdir(os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia))
        print('AVISO: No existen boletines para la fecha ', current_date.strftime('%Y-%m-%d'))
    else:
        xmls = xml_sumario.getElementsByTagName('urlXml')
        for xml in xmls:
            fichero_xml = PATH_DATA + '\\' + str(fecha_anno) + '\\' + str(fecha_mes) + '\\' + str(fecha_dia) + '\\' + 'xmls' + '\\' + xml.firstChild.nodeValue.split('=')[-1] + '.xml'
            #fichero_xml = os.path.join(PATH_DATA, fecha_anno, fecha_mes, fecha_dia, 'xmls', xml.firstChild.nodeValue.split('=')[-1] + '.xml').replace(' ', '_').replace('/', '\\')[1:]

            if os.path.exists(fichero_xml):
                print("fichero encontrado")
                continue


            print('Solicitando', f'{BOE_URL}{xml.firstChild.nodeValue} --> {fichero_xml}')
            intentos = 0
            max_intentos = 5
            while intentos < max_intentos:
                if intentos != 0:
                    import time
                    time.sleep(5)
                    print(f'Intento {intentos}')

                traer_documento(f'{BOE_URL}{xml.firstChild.nodeValue}', fichero_xml)
                intentos += 1

                if os.path.getsize(fichero_xml):
                    break

            if not os.path.getsize(fichero_xml):
                print('ERROR: El tamaño del fichero xml descargado no coincide con el del XML del Sumario '
                    f'(Descargado: {os.getsize(fichero_xml)} <> XML: {fichero_xml_tamano_xml})')
                sys.exit(1)

    if semaphore is not None:
        semaphore.release()

In [6]:
current_date = START_DATE
hoy = datetime.now()
while current_date <= hoy and 0:
    get_xmls(current_date)
    get_pdfs(current_date)
    current_date += diff_1_day

In [7]:
n_hilos = 6
semaphore = threading.Semaphore(n_hilos)
dates =  [START_DATE + timedelta(days=i) for i in range((datetime.now() - START_DATE).days + 1)]
hilos = []

print(dates)

for date in dates:
    semaphore.acquire()

    print('current_date:', date.strftime('%Y-%m-%d'))
    
    hilo = threading.Thread(target=get_xmls, args=(date, semaphore))
    #threading.Thread(target=get_pdfs, args=(date,)).start()
    hilos.append(hilo)
    hilo.start()

for hilo in hilos:
    hilo.join()


[datetime.datetime(2023, 10, 1, 0, 0), datetime.datetime(2023, 10, 2, 0, 0), datetime.datetime(2023, 10, 3, 0, 0), datetime.datetime(2023, 10, 4, 0, 0), datetime.datetime(2023, 10, 5, 0, 0), datetime.datetime(2023, 10, 6, 0, 0), datetime.datetime(2023, 10, 7, 0, 0), datetime.datetime(2023, 10, 8, 0, 0), datetime.datetime(2023, 10, 9, 0, 0), datetime.datetime(2023, 10, 10, 0, 0), datetime.datetime(2023, 10, 11, 0, 0), datetime.datetime(2023, 10, 12, 0, 0), datetime.datetime(2023, 10, 13, 0, 0), datetime.datetime(2023, 10, 14, 0, 0), datetime.datetime(2023, 10, 15, 0, 0), datetime.datetime(2023, 10, 16, 0, 0), datetime.datetime(2023, 10, 17, 0, 0), datetime.datetime(2023, 10, 18, 0, 0), datetime.datetime(2023, 10, 19, 0, 0), datetime.datetime(2023, 10, 20, 0, 0), datetime.datetime(2023, 10, 21, 0, 0), datetime.datetime(2023, 10, 22, 0, 0), datetime.datetime(2023, 10, 23, 0, 0), datetime.datetime(2023, 10, 24, 0, 0), datetime.datetime(2023, 10, 25, 0, 0), datetime.datetime(2023, 10, 26, 0

# Almacenamieto de datos

In [8]:
def proceso_sumario(path_xml):
    data = {}
    xml_sumario = minidom.parse(path_xml)

    anno, mes, dia = path_xml.split('\\')[2:-2]
    file_name = path_xml.split('\\')[-1].split('.')[0]
    date = path_xml.split('\\')[-1].split('.')[0].split('-')[-2]

    data['index'] = file_name.replace(date, f'{anno}{mes}{dia}')

    aux = {}
    documento = xml_sumario.documentElement
    i = 0
    for element in documento.getElementsByTagName('*'):
        #print("Elemento:", element.tagName)
        if element.tagName == 'p': element.tagName = element.getAttribute('class') + '_' + str(i); i += 1
        data[element.tagName] = {}
        for attr_name, attr_value in element.attributes.items():
            data[element.tagName][attr_name] = attr_value
            #print(f"Atributo: {attr_name} - Valor: {attr_value}")
        if element.firstChild and element.firstChild.nodeType == element.TEXT_NODE:
            data[element.tagName]['Contenido'] = element.firstChild.data
            #print("Contenido:", element.firstChild.data)

    data_to_save = {}
    keys_to_save = ['identificador', 'index', 'origen_legislativo', 'departamento', 'titulo', 'fecha_publicacion']
    #keys_to_save = keys_to_save.extend([i for i in data.keys() if i.startswith('parrafo')])
    keys_to_save.extend([e for i, e in enumerate(data.keys()) if e.startswith('parrafo') or e.startswith('articulo')])
    for key in keys_to_save:
        try:
            if key == 'index':
                data_to_save[key] = data[key]
            elif key.startswith('parrafo'):
                data_to_save[key] = data[key]['Contenido'].replace('\xa0', ' ').replace('\u2003', ' ')
            else:
                data_to_save[key] = data[key]['Contenido'].replace('\xa0', ' ').replace('\u2003', ' ')
        except Exception as e:
            pass
            #print(f"{path_xml} excepcion: {e}")
    
    return data_to_save

In [9]:
def process_data():
    data = {}
    i = 0
    #itero por las carpetas de boe
    for anio in os.listdir('boe/dias'):
        #itero por los archivos de cada carpeta
        for mes in os.listdir(f'boe/dias/{anio}'):
            #si el archivo es un pdf
            for dia in os.listdir(f'boe/dias/{anio}/{mes}'):
                path_xmls = os.path.join('boe', 'dias', anio, mes, dia, 'xmls')
                print(path_xmls)
                for xml in os.listdir(path_xmls):
                    path_xml = os.path.join(path_xmls, xml)
                    file_data = proceso_sumario(path_xml)
                    data[file_data['identificador']] = file_data
                    i += 1
                    #if i > 10: break
    
    return data

In [10]:
data = process_data()
data['BOE-A-2023-20397']

boe\dias\2023\10\02\xmls
boe\dias\2023\10\03\xmls
boe\dias\2023\10\04\xmls
boe\dias\2023\10\05\xmls
boe\dias\2023\10\06\xmls
boe\dias\2023\10\07\xmls
boe\dias\2023\10\09\xmls
boe\dias\2023\10\10\xmls
boe\dias\2023\10\11\xmls
boe\dias\2023\10\12\xmls
boe\dias\2023\10\13\xmls
boe\dias\2023\10\14\xmls
boe\dias\2023\10\16\xmls
boe\dias\2023\10\17\xmls
boe\dias\2023\10\18\xmls
boe\dias\2023\10\19\xmls
boe\dias\2023\10\20\xmls
boe\dias\2023\10\21\xmls
boe\dias\2023\10\23\xmls
boe\dias\2023\10\24\xmls
boe\dias\2023\10\25\xmls
boe\dias\2023\10\26\xmls
boe\dias\2023\10\27\xmls
boe\dias\2023\10\28\xmls
boe\dias\2023\10\30\xmls
boe\dias\2023\10\31\xmls
boe\dias\2023\11\01\xmls
boe\dias\2023\11\02\xmls
boe\dias\2023\11\03\xmls
boe\dias\2023\11\04\xmls


{'identificador': 'BOE-A-2023-20397',
 'index': 'BOE-A-20231002-20397',
 'origen_legislativo': 'Estatal',
 'departamento': 'Cortes Generales',
 'titulo': 'Resolución de 21 de septiembre de 2023, de las Presidencias del Congreso de los Diputados y del Senado, por la que se nombra personal funcionario del Cuerpo de Archiveros-Bibliotecarios de las Cortes Generales.',
 'fecha_publicacion': '20231002',
 'parrafo_0': 'En uso de las atribuciones que nos confiere el artículo 14.b) del Estatuto del Personal de las Cortes Generales, y a propuesta del Tribunal que ha juzgado la oposición convocada el 9 de marzo de 2022, nombramos funcionarios del Cuerpo de Archiveros-Bibliotecarios de las Cortes Generales, a los opositores que a continuación y por el orden de puntuación obtenida se relacionan:',
 'parrafo_2_1': '1. Doña Clara Espartero Guio.',
 'parrafo_2': '2. Don Óscar Raúl Donaire Bravo.',
 'parrafo_2_3': 'Madrid, 21 de septiembre de 2023.–La Presidenta del Congreso de los Diputados, Francina

In [11]:
#!pip install elasticsearch

In [14]:
from elasticsearch import Elasticsearch
import json

#client = Elasticsearch("http://elasticsearch:9200")
client = Elasticsearch("http://localhost:9200")

## Crear index

In [ ]:
config = {
    "mappings": {
        "properties": {
           "text": {"type": "text", "similarity": "BM25"},
            "embeddings": {
                "type": "dense_vector",
                "dims": 512,
                "index": True,
                "similarity": "cosine"
            }
        }
    },
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    }
}

In [18]:
for key in data.keys():
    print(key)
    fecha = data[key]['fecha_publicacion']
    id = key
    document = data[key]
    try:
        client.index(
            index = fecha,
            id=id,
            document=document
        )
    except Exception as e:
        print(e)
        print(f"len: {len(document)}")
        print(fecha)
        print(id)
        print(document)
        break

BOE-A-2023-20397
BOE-A-2023-20398
BOE-A-2023-20399
BOE-A-2023-20400
BOE-A-2023-20401
BOE-A-2023-20402
BOE-A-2023-20403
BOE-A-2023-20404
BOE-A-2023-20405
BOE-A-2023-20406
BOE-A-2023-20407
BOE-A-2023-20408
BOE-A-2023-20409
BOE-A-2023-20410
BOE-A-2023-20411
BOE-A-2023-20412
BOE-A-2023-20413
BOE-A-2023-20414
BOE-A-2023-20415
BOE-A-2023-20416
BOE-A-2023-20417
BOE-A-2023-20418
BOE-A-2023-20419
BOE-A-2023-20420
BOE-A-2023-20421
BOE-A-2023-20422
BOE-A-2023-20423
BOE-A-2023-20424
BOE-A-2023-20425
BOE-A-2023-20426
BOE-A-2023-20427
BOE-A-2023-20428
BOE-A-2023-20429
BOE-A-2023-20430
BOE-A-2023-20431
BOE-A-2023-20432
BOE-A-2023-20433
BOE-A-2023-20434
BOE-A-2023-20435
BOE-A-2023-20436
BOE-A-2023-20437
BOE-A-2023-20438
BOE-A-2023-20439
BOE-A-2023-20440
BOE-A-2023-20441
BOE-A-2023-20442
BOE-A-2023-20443
BOE-A-2023-20444
BOE-A-2023-20445
BOE-A-2023-20446
BOE-A-2023-20447
BOE-A-2023-20448
BOE-A-2023-20449
BOE-A-2023-20450
BOE-A-2023-20451
BOE-A-2023-20452
BOE-A-2023-20453
BOE-A-2023-20454
BOE-A-2023-204

In [17]:
data['BOE-A-2023-20468']

{'identificador': 'BOE-A-2023-20468',
 'index': 'BOE-A-20231002-20468',
 'origen_legislativo': 'Estatal',
 'departamento': 'Universidades',
 'titulo': 'Resolución de 15 de septiembre de 2023, de la Universidad de A Coruña, por la que se convoca concurso de acceso a plaza de cuerpos docentes universitarios.',
 'fecha_publicacion': '20231002',
 'parrafo_0': 'De conformidad con lo dispuesto en la Ley Orgánica 2/2023, de 22 de marzo (BOE del 23), del Sistema Universitario, en adelante (LOSU), en el Real Decreto 1312/2007, de 5 de octubre, por el que se establece la acreditación nacional para el acceso a los cuerpos docentes universitarios (en adelante RDAN) y en el Real Decreto 1313/2007, de 5 de octubre, por el que se regula el régimen de los concursos de acceso a cuerpos docentes universitarios (en adelante RDCA), y a tenor de lo establecido en los Estatutos de la Universidad de A Coruña, aprobados por el Decreto 101/2004, de 13 de mayo (DOG de 26 de mayo) y modificados por el Decreto 19